In [2]:
import boto3
import pandas as pd
import os
import pandas as pd
from bs4 import BeautifulSoup
from botocore.exceptions import ClientError

In [3]:
s3 = boto3.client('s3')

In [6]:
files = []
for ew in ['W','E']:
    for lon in range(0,2):
        for ns in ['S','N']:
            for lat in range(0,2):
                files.append(
                    ns + f"{lat}".rjust(2,'0') + "_00_" + ew + f"{lon}".rjust(3,'0') + "_00"
                )
files = pd.Series(files).sample(frac=1)
print(len(files))
files[:5]

16


1    S01_00_W000_00
3    N01_00_W000_00
2    N00_00_W000_00
7    N01_00_W001_00
8    S00_00_E000_00
dtype: object

In [7]:
dir_prefix = 'Copernicus_DSM_COG_30_'
dir_suffix = '_DEM'
bucket = 'copernicus-dem-90m'

file_record = pd.read_csv('copernicus_files.csv', names=['file','status','lat','lon','min','max','mean','std'])
files_processed = set(file_record.file)

for f in files.sample(frac=1):
    path = dir_prefix + f + dir_suffix + '/'
    file = 'Copernicus_DSM_30_' + f + '.xml' 

    output = 'global_elevation_data/' +file

    if output not in files_processed:
        if not os.path.exists(output):
            try: 
                s3.download_file(bucket, path+file, output)
            except ClientError:
                print(".", end="")
                with open('copernicus_files.csv', 'a+') as copernicus_record:
                    copernicus_record.write(output + ",0,,,,,,"+"\n")
        else:
            with open(output,'r') as open_file:
                soup = BeautifulSoup(open_file, 'xml')
                row = [output,'1',
                       soup.find_all('tsxx:upperLeftLatitude')[0].text,
                       soup.find_all('tsxx:upperLeftLongitude')[0].text,
                       soup.find_all('tsxx:minValue')[0].text,
                       soup.find_all('tsxx:maxValue')[0].text,
                       soup.find_all('tsxx:meanValue')[0].text,
                       soup.find_all('tsxx:stdDev')[0].text
                      ]
                with open('copernicus_files.csv', 'a+') as copernicus_record:
                    copernicus_record.write(",".join(row) + "\n")
                    print("!", end="")

..........